#### Import standard Python modules

In [2]:
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

#### Import Azure ML SDK modules

In [13]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

In [12]:
#!pip install azureml.core

  Using cached https://files.pythonhosted.org/packages/8b/59/e9fb3bf15c4a0d47d5c6b842df21923049d7b2d16d9a106f95a0cdf4c4b7/azureml_core-1.32.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bb/23/00fe4fbf9963f3bcb34a443eba0d0283fc51e5887d4045552c87490394e4/pywin32-227-cp37-cp37m-win_amd64.whl
  Found existing installation: pywin32 301
    Uninstalling pywin32-301:
      Successfully uninstalled pywin32-301


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\91982\\appdata\\roaming\\python\\python37\\~ite-packages\\pywin32_system32\\pywintypes37.dll'
Consider using the `--user` option or check the permissions.



In [11]:
#!pip install --ignore-installed pywin32 --user

  Using cached https://files.pythonhosted.org/packages/84/e0/f7c34be2e080c248e378109c3a848cd07ab8e45e11d8dc9f1d818b3940c7/pywin32-301-cp37-cp37m-win_amd64.whl


ERROR: docker 4.4.4 has requirement pywin32==227; sys_platform == "win32", but you'll have pywin32 301 which is incompatible.


#### Check Azure ML SDK version

In [14]:
print(azureml.core.VERSION)

1.32.0


#### Create Azure ML Workspace

In [15]:
AZ_SUBSCRIPTION_ID='2ea63820-da0b-4025-b108-b3b13120a031'
ws = Workspace.create(name='salary',
                      subscription_id=AZ_SUBSCRIPTION_ID, 
                      resource_group='akash_salary_azure',
                      create_resource_group=True,
                      location='southeastasia'
                     )

Performing interactive authentication. Please follow the instructions on the terminal.


Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


Deploying StorageAccount with name salarystoragec94359ee0bf.
Deploying KeyVault with name salarykeyvaultd052138629.
Deploying AppInsights with name salaryinsights6a544ae8ff.
Deployed AppInsights with name salaryinsights6a544ae8ff. Took 11.02 seconds.
Deployed KeyVault with name salarykeyvaultd052138629. Took 19.9 seconds.
Deployed StorageAccount with name salarystoragec94359ee0bf. Took 24.67 seconds.
Deploying Workspace with name salary.
Deployed Workspace with name salary. Took 98.31 seconds.


#### Write configuration to local file

In [16]:
ws.write_config()

In [17]:
ws.get_details()

{'id': '/subscriptions/2ea63820-da0b-4025-b108-b3b13120a031/resourceGroups/akash_salary_azure/providers/Microsoft.MachineLearningServices/workspaces/salary',
 'name': 'salary',
 'identity': {'principal_id': 'e0b88de8-54bc-4eae-bec9-d1a623ffacc2',
  'tenant_id': 'fa57b8ce-4d4c-432b-964f-a20d21d7a6a4',
  'type': 'SystemAssigned'},
 'location': 'southeastasia',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '6296ec3c-5256-4bf3-9b43-f8ecb2ee4a21',
 'sdkTelemetryAppInsightsKey': '0855780c-10d5-4461-ae3c-9b15eb18c90d',
 'description': '',
 'friendlyName': 'salary',
 'creationTime': '2021-08-02T07:54:58.1308570+00:00',
 'keyVault': '/subscriptions/2ea63820-da0b-4025-b108-b3b13120a031/resourcegroups/akash_salary_azure/providers/microsoft.keyvault/vaults/salarykeyvaultd052138629',
 'applicationInsights': '/subscriptions/2ea63820-da0b-4025-b108-b3b13120a031/resourcegroups/akash_salary_azure/providers/microsoft.insights/components/salaryinsi

#### Create Azure ML Experiment

In [18]:
exp = Experiment(workspace=ws, name='salexp')

Experiments is a mechanism to track metrics. We will have to perform experiments (iterate many times) before getting the final model. The tuning of paramaters like epochs, rtc can be logged in Azure and we can visualize and refer these logs for our purpose.

#### Start logging metrics

In [19]:
run = exp.start_logging()                   
run.log("Experiment start time", str(datetime.datetime.now()))

#### Load salary dataset

In [20]:
sal = pd.read_csv('data/sal.csv',header=0, index_col=None)
X = sal[['x']]
y = sal['y']

#### Split the train and test data

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

#### Train the model

In [22]:
lm = LinearRegression() 
lm.fit(X_train,y_train) 

LinearRegression()

#### Freeze the model

In [23]:
filename = 'outputs/sal_model.pkl'
joblib.dump(lm, filename)

['outputs/sal_model.pkl']

#### Test the model

In [24]:
filename = 'outputs/sal_model.pkl'
loaded_model=joblib.load(filename)
y=loaded_model.predict([[21]])[0]
print(y)

141760.56910569107


#### Log metrics to Azure ML Experiment

In [25]:
run.log('Intercept :', lm.intercept_)
run.log('Slope :', lm.coef_[0])

#### End Azure ML Experiment

In [26]:
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

#### Get Portal URL

In [27]:
print(run.get_portal_url())

https://ml.azure.com/runs/d4857852-d512-4eb6-80b2-34d84492c3b4?wsid=/subscriptions/2ea63820-da0b-4025-b108-b3b13120a031/resourcegroups/akash_salary_azure/workspaces/salary&tid=fa57b8ce-4d4c-432b-964f-a20d21d7a6a4


#### Register the model

In [28]:
model = Model.register(model_path = "outputs/sal_model.pkl",
                       model_name = "sal_model",
                       tags = {"key": "1"},
                       description = "Salary Prediction",
                       workspace = ws)

Registering model sal_model


#### Define Azure ML Deploymemt configuration

In [29]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Salary",  "method" : "sklearn"}, 
                                               description='Predict Stackoverflow Salary')

ACI is Azure Constainer Instance to deploy our model into Azure. It is serverless container infrastructure instance on Azure. 

#### Create enviroment configuration file

salenv.yml is to specify the dependencies which are required to be installed in Azure for running the model. Azure ML parses the yml file and gets all the version dependencies to be installed.

In [30]:
salenv = CondaDependencies()
salenv.add_conda_package("scikit-learn")

with open("salenv.yml","w") as f:
    f.write(salenv.serialize_to_string())
with open("salenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



#### Create Azure ML Scoring file

In [31]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('sal_model')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score.py


#### Deploy the model to Azure Container Instance


In [32]:
%%time
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="salenv.yml")

Wall time: 0 ns


C:\Users\91982\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  This is separate from the ipykernel package so we can avoid doing imports until


#### We have created a Docker Image with 3 files -
1. Pickle file 
2. YML file
3. Score.py - scoring file

#### This Docker Image needs to be exposed so we need to run the Docker image and expose it to outside users as a web service. 

#### Expose web service

In [34]:
service = Webservice.deploy_from_model(workspace=ws,
                                       name='salary-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

C:\Users\91982\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: deploy_from_model has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  """


Creating image
Running..................................................................................
Succeeded
Image creation operation finished for image salary-svc:2, operation "Succeeded"


WebserviceException: WebserviceException:
	Message: Service salary-svc with the same name already exists, please use a different service name or delete the existing service.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service salary-svc with the same name already exists, please use a different service name or delete the existing service."
    }
}

#### Get the Web Service URL

In [23]:
print(service.scoring_uri)

http://40.119.206.56:80/score


In [26]:
!curl -X POST \
	-H 'Content-Type':'application/json' \
	-d '{"data":[[45]]}' \
	http://40.119.206.56:80/score

"[185924.7967479675]"

#### Delete Workspace and clean up resources

In [35]:
ws.delete()